<a href="https://colab.research.google.com/github/genomelab/esFunctions/blob/master/1_Segment_Image_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install DeepCell
#pip install git+https://github.com/angelolab/mibi-bin-tools.git@v0.2.5
#pip install ark-analysis
#pip install imagecodecs
#pip install ark-utils
#pip install numpy==1.23.4

### Restart the Runtime after installing modules.
### This is a notebook to format your data for segmentation, run the images through the cloud instance of Mesmer, and then extract marker counts and morphological information from all the cells in your images

In [ ]:
#name of file to download and name of the project
# It also creates a small version of the files in proejct_name with small added to it
filename = 'NBL_N0351_Scan1.qptiff'
project_name = 'neuroblastoma'
project_base_dir = "../data/" + project_name


project_name_small = project_name + "_small"
project_small_base_dir = "../data/" + project_name_small

#coordinates of image to extract for small image
x1 = 20000
y1 = 25000
x2 = 20500
y2 = 25500


In [ ]:
# import required packages
# import required packages
import os
import warnings

import matplotlib.pyplot as plt
import numpy as np
import skimage.io as io
import xarray as xr
from alpineer import io_utils

from ark.segmentation import marker_quantification, segmentation_utils
from ark.utils import (deepcell_service_utils, example_dataset,
                       plot_utils)
import imagecodecs

## 0: Set root directory and download example dataset

Here we are using the example data located in `/data/example_dataset/input_data`. To modify this notebook to run using your own data, simply change `base_dir` to point to your own sub-directory within the data folder.

* `base_dir`: the path to all of your imaging data. This directory will contain all of the data generated by this notebook.

In [ ]:
## This works only with google colab on google machines if runningn locally need to connect via pydrive 
#from google.colab import drive
#drive.mount('/content/drive')

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from pydrive2.fs import GDriveFileSystem

gauth = GoogleAuth()
gauth.LocalWebserverAuth()



If you would like to test the features in Ark with an example dataset, run the cell below. It will download a dataset consisting of 11 FOVs with 22 channels. You may find more information about the example dataset in the [README](../README.md#example-dataset).

If you are using your own data, skip the cell below.

* `overwrite_existing`: If set to `False`, it will not overwrite existing data in the `data/example_dataset`. Recommended leaving as `True` if you are doing a clean run of the `ark` pipeline using this dataset from the start. If you already have the dataset downloaded, set to `False`.

## 1: set file paths and parameters

### All data, images, files, etc. must be placed in the 'data' directory, and referenced via '../data/path_to_your_data'

If you're interested in directly interfacing with Google Drive, consult the documentation [here](https://ark-analysis.readthedocs.io/en/latest/_rtd/google_docs_usage.html).

In [ ]:
# set up file paths
# set up the base directory
base_dir = projcet_base_dir
cell_table_dir = os.path.join(base_dir, "segmentation/cell_table")
deepcell_input_dir = os.path.join(base_dir, "segmentation/deepcell_input")
deepcell_output_dir = os.path.join(base_dir, "segmentation/deepcell_output")
deepcell_visualization_dir = os.path.join(base_dir, "segmentation/deepcell_visualization")

# create directories if do not exist
for directory in [cell_table_dir, tiff_dir, deepcell_input_dir, deepcell_output_dir, deepcell_visualization_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# validate paths
io_utils.validate_paths([base_dir,
                         tiff_dir,
                         deepcell_input_dir,
                         deepcell_output_dir,
                         cell_table_dir,
                         deepcell_visualization_dir
                         ])

In [ ]:
# Download the image file 
drive = GoogleDrive(gauth)

#file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
#for file1 in file_list:
#  print('title: %s, id: %s, Mimetype: %s' % (file1['title'], file1['id'], file1['mimeType']))

# Image file name


download_file = os.path.join(base_dir, filename)

# Query
query1 = {'q' : f"mimeType = 'image/tiff'"}
query2 = {'q': f"title = '{filename}'"}

# Get list of files that match against the query1 for tiff files if needed
#tiff_files = drive.ListFile(query1).GetList()
#for file1 in tiff_files:
#  print('List of Files: %s, id: %s, Mimetype: %s' % (file1['title'], file1['id'], file1['mimeType']))

myfiles = drive.ListFile(query2).GetList()
print('File to download: %s, id: %s, Mimetype: %s' % (myfiles[0]['title'], myfiles[0]['id'], myfiles[0]['mimeType']))


img_file = drive.CreateFile(myfiles[0])
img_file.GetContentFile(download_file, remove_bom=True) 
## fsspec connection did not work.  abandoened it
#fs = GDriveFileSystem("root", gauth)
#fs.listdir("root")
#f = fs.open('root/NBL_N0351_Scan1.qptiff')


In [ ]:
import tifffile as tif

### Shahab Asgharzadeh - Feb 2023
### Takes OME-TIFF or QPTIFF file and seperates channels and saves them in a fov folder under tiff_dir folder 
### - optional create square chunks given a size (max for DeepLearn Kiosk is 2048x2048) 
### - saves the chunks under seperate fov files (fov0, fov1, etc.)
### - images are saved based on channel names provided (order needs to be correct)


chunk_size = 0
channel_names = [ 'DAPI', 'CD45R0', 'CD31', 'CD4', 'CD8', 'CD20', 'CD3', 'Ki67','PanCK', 
'PCNA', 'CD163', 'PDL1', 'CD14']


def convert_ome_tiff_to_channels(input_file, chunk_size):
    # Open the OME-TIFF file and get the number of channels
    with tif.TiffFile(input_file) as tif_file:
        channel_count = tif_file.series[0].shape[0]

    # Loop over each channel and save it as a separate TIFF file
    # If Chunk size other than 0 given, it will create regions of interest as square for
    # entire image for each channel in seperate directories. 
    
    for channel_index in range(channel_count):
        # Get the output filename for the current channel
        if chunk_size == 0:
            FOV_dir = os.path.join(tiff_dir, "fov0")
            if not os.path.exists(FOV_dir):
                os.makedirs(FOV_dir)
                io_utils.validate_paths([FOV_dir])
            output_filename = os.path.join(FOV_dir,(f'{channel_names[channel_index]}.tiff'))

            # Open the OME-TIFF file and extract the image data for the current channel
            with tif.TiffFile(input_file) as tif_file:
                channel_image_data = tif_file.asarray(key=channel_index)

            # Save the image data as a TIFF file
            tif.imwrite(output_filename, channel_image_data)

            print(f'Saved channel {channel_index} to {output_filename}')
        else: 
            # Open the OME-TIFF file and extract the image data for the current channel
            # and create chunks of that channel and place in different fov folders
            with tif.TiffFile(input_file) as tif_file:
                channel_image_data = tif_file.asarray(key=channel_index)
                k = 0
                height, width = channel_image_data.shape
                chunks = []
                for i in range(0, height, chunk_size):
                    for j in range(0, width, chunk_size):
                        chunk = channel_image_data[i:i+chunk_size, j:j+chunk_size]
                        if chunk.shape[0] == chunk_size and chunk.shape[1] == chunk_size:
                            FOV_dir = os.path.join(tiff_dir, f"fov{k}")
                            k=k+1                    
                            if not os.path.exists(FOV_dir):
                                os.makedirs(FOV_dir)
                            output_filename = os.path.join(FOV_dir,(f'{channel_names[channel_index]}.tiff'))
                            tif.imwrite(output_filename, chunk)
            print('Saved channels and chunks in fov subdirectories')
        
convert_ome_tiff_to_channels(download_file, chunk_size)


In [ ]:
# setup directory for smaller version and take each tiff from fov0 and generate 
# smaller version of the file mainly for testing and visualization
# regions selected in the top cell

# set up file paths
# set up the base directory
base_dir = project_small_base_dir
tiff_dir = os.path.join(base_dir, "image_data")
cell_table_dir = os.path.join(base_dir, "segmentation/cell_table")
deepcell_input_dir = os.path.join(base_dir, "segmentation/deepcell_input")
deepcell_output_dir = os.path.join(base_dir, "segmentation/deepcell_output")
deepcell_visualization_dir = os.path.join(base_dir, "segmentation/deepcell_visualization")

# create directories if do not exist
for directory in [cell_table_dir, tiff_dir, deepcell_input_dir, deepcell_output_dir, deepcell_visualization_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# validate paths
io_utils.validate_paths([base_dir,
                         tiff_dir,
                         deepcell_input_dir,
                         deepcell_output_dir,
                         cell_table_dir,
                         deepcell_visualization_dir
                         ])


# Define the input and output directories
input_dir = project_base_dir
output_dir = project_small_base_dir


# Loop through each subdirectory in the input directory
for root, dirs, files in os.walk(input_dir):
    # Create the corresponding subdirectory in the output directory
    output_subdir = os.path.join(output_dir, os.path.relpath(root, input_dir))
    os.makedirs(output_subdir, exist_ok=True)

    # Loop through each TIFF file in the subdirectory
    for file in files:
        if file.endswith('.tif') or file.endswith('.tiff'):
            # Open the TIFF file and extract the region of interest
            input_file = os.path.join(root, file)
            with tifffile.TiffFile(input_file) as tif:
                img = tif.asarray()
                region = img[y1:y2, x1:x2]

            # Save the extracted region to a new file in the output directory
            output_file = os.path.join(output_subdir, file)
            tifffile.imwrite(output_file, region)




In [ ]:
## For testing set the base directory to project_small_dir
##  otherwise comment it out
base_dir = project_base_dir
base_dir = project_small_base_dir
tiff_dir = os.path.join(base_dir, "image_data")
cell_table_dir = os.path.join(base_dir, "segmentation/cell_table")
deepcell_input_dir = os.path.join(base_dir, "segmentation/deepcell_input")
deepcell_output_dir = os.path.join(base_dir, "segmentation/deepcell_output")
deepcell_visualization_dir = os.path.join(base_dir, "segmentation/deepcell_visualization")

### Compute and filter fov paths

In [ ]:
# either get all fovs in the folder...
fovs = io_utils.list_folders(tiff_dir)

# ... or optionally, select a specific set of fovs manually
#prefix = "fov"
#fovs = [f"{prefix}{i}" for i in range(242,389)]
##fovs = ["fov0", "fov1"]
print(fovs)


### Load images into notebook, process, and save as Mesmer compatable input

In [ ]:
# NOTE: at least one of nucs and mems must not be None
# nuclear channel name(s) (or nucs = None)
nucs = ['DAPI']

# membrane channel name(s) (or mems = None)
mems = ['CD3', 'CD14']

In [ ]:
# generate and save deepcell input tiffs
# set img_sub_folder param to None if the image files in tiff_dir are not in a separate sub folder 
deepcell_service_utils.generate_deepcell_input(
    deepcell_input_dir,
    tiff_dir,
    nucs,
    mems,
    fovs,
    img_sub_folder=None
)

In [ ]:
#prepare the image for mesmer

from skimage.io import (imread, imsave)
input_file = os.path.join(deepcell_input_dir, 'fov0.tiff')

im = imread(input_file)

im1 = im[0]
im2 = im[1]
im = np.stack((im1, im2), axis = -1)
im = np.expand_dims(im,0)




In [ ]:
from deepcell.applications import Mesmer
from skimage.io import (imread, imsave)

image_to_predict = im

app = Mesmer()
labeled_image = app.predict(image_to_predict)
np.save(os.path.join(deepcell_output_dir, 'fov0.npy'), labeled_image)

In [ ]:
from deepcell.utils.plot_utils import create_rgb_image
rgb_images = create_rgb_image(image_to_predict, channel_colors=['green', 'blue'])


In [ ]:
from matplotlib import pyplot as plt

# select index for displaying
idx = 0

# plot the data
fig, ax = plt.subplots(1, 3, figsize=(15, 15))
ax[0].imshow(image_to_predict[idx, ..., 0])
ax[1].imshow(image_to_predict[idx, ..., 1])
ax[2].imshow(rgb_images[idx, ...])

ax[0].set_title('Nuclear channel')
ax[1].set_title('Membrane channel')
ax[2].set_title('Overlay')

plt.show()

In [ ]:
from deepcell.utils.plot_utils import make_outline_overlay
overlay_data = make_outline_overlay(rgb_data=rgb_images, predictions=labeled_image)

In [ ]:
# select index for displaying
idx = 0

# plot the data
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(rgb_images[idx, ...])
ax[1].imshow(overlay_data[idx, ...])
ax[0].set_title('Raw data')
ax[1].set_title('Predictions')
plt.show()

## 2: Upload files to Deepcell and download results

Deepcell input images will be zipped into a single file, uploaded to [deepcell.org](https://deepcell.org),

and the output will be downloaded to the deepcell output directory.

In [ ]:
# Mesmer was trained on data acquired at 20X resolution. If your image data was acquired at a different resolution, you will get the best performance by rescaling. The rescale factor will increase or decrease the image resolution by the value you provide. For example, if you data was acquired at 10X, use a `rescale_factor` of 2. If your data was acquired at 60X resolution, use a `rescale_factor` of 0.33.
#rescale_factor = 1.0

In [ ]:
## Won't use the Kiosk

#deepcell_service_utils.create_deepcell_output(deepcell_input_dir, deepcell_output_dir, fovs=fovs, scale=rescale_factor)

### We can then save the segmented mask overlaid on the imaging data

In [ ]:
# write OME-TIFF
from tifffile import imwrite
imwrite(os.path.join(deepcell_output_dir, 'fov0_whole_cell.tiff'), overlay_data)
#np.save(os.path.join(deepcell_output_dir, 'fov0_whole_cell.tiff'), labeled_image)

In [ ]:
# display the channel overlay for a fov, useful for quick verification
warnings.simplefilter("ignore")

fov_to_display = io_utils.remove_file_extensions([fovs[0]])[0]

fov_overlay = plot_utils.create_overlay(
    fov=fov_to_display,
    segmentation_dir=deepcell_output_dir,
    data_dir=deepcell_input_dir,
    img_overlay_chans=['nuclear_channel', 'membrane_channel'],
    seg_overlay_comp='whole_cell'
)

_ = io.imshow(fov_overlay)

In [ ]:
print(fov_to_display)

In [ ]:
# save the overlaid segmentation labels for each fov (these will not display, but will save in viz_dir)
segmentation_utils.save_segmentation_labels(
    segmentation_dir=deepcell_output_dir,
    data_dir=deepcell_input_dir,
    output_dir=deepcell_visualization_dir,
    fovs=io_utils.remove_file_extensions(fovs),
    channels=['nuclear_channel', 'membrane_channel']
)

### Afterwards, we can generate expression matrices from the labeling + imaging data

In [ ]:
# set to True to add nuclear cell properties to the expression matrix
nuclear_counts = True

For a full list of features extracted, please refer to the cell table section of: https://ark-analysis.readthedocs.io/en/latest/_rtd/data_types.html

In [ ]:
# now extract the segmented imaging data to create normalized and transformed expression matrices
# note that if you're loading your own dataset, please make sure all the imaging data is in the same folder
# with each fov given its own folder and all fovs having the same channels
cell_table_size_normalized, cell_table_arcsinh_transformed = \
    marker_quantification.generate_cell_table(segmentation_dir=deepcell_output_dir,
                                              tiff_dir=tiff_dir,
                                              img_sub_folder=None,
                                              fovs=fovs,
                                              batch_size=5,
                                              nuclear_counts=nuclear_counts)

In [ ]:
# Set the compression level if desired, ZSTD compression can offer up to a 60-70% reduction in file size.
# NOTE: Compressed `csv` files cannot be opened in Excel. They must be uncompressed beforehand.
compression = None

# Uncomment the line below to allow for compressed `csv` files.
# compression = {"method": "zstd", "level": 3}

cell_table_size_normalized.to_csv(os.path.join(cell_table_dir, 'cell_table_size_normalized.csv'),
                                  compression=compression, index=False)
cell_table_arcsinh_transformed.to_csv(os.path.join(cell_table_dir, 'cell_table_arcsinh_transformed.csv'),
                                      compression=compression, index=False)